## Lab - Customizing Large Language Models with LangChain

### Introduction

Welcome to the LLM Customization Lab! In this activity, you'll explore how to customize and control **Large Language Models (LLMs)** to create specialized AI assistants.

**What you'll learn:**
- How to interact with language models using LangChain
- How to customize AI behavior with system prompts
- How to inject custom knowledge into an AI assistant
- How to create and test your own custom AI assistants

**By the end of this lab**, you'll have built multiple custom AI assistants, each with unique personalities and knowledge!

### Part 0 - Background Research

Before diving into the code, let's explore the concepts behind Large Language Models and AI customization.

To answer the questions, edit the markdown cell and put your answer below the question.

**Make sure to save the markdown cell by pressing the ✓ (check) icon in the top right after answering the questions**

##### Question 00
What is a Large Language Model (LLM)? How is it different from traditional software?
- **Answer:** A Large Langugae Model is an artificial intelligence system trained on massive amounts of text so it can understand and generate human-like language

##### Question 01
What does it mean to "prompt" an LLM? Why is prompting important?
- **Answer:** It means you have to give a set of instructions to an LLM to get a specific response

##### Question 02
Research "prompt engineering." What are some techniques for getting better responses from LLMs?
- **Answer:** Some techniques  for getting better responses from LLMs is providing examples and breaking down the task into steps.

##### Question 03
What are some ethical concerns with customizing AI behavior?
- **Answer:** Customizing AI can lead to biased, misleading, or harmful outputs.

### Part 1 - Setting Up Our Environment

First, we need to install and import the libraries we'll use to work with Large Language Models.

#### 1.0 - Installing Required Libraries

Before we can import our libraries, we need to make sure they're installed. Run these commands in your terminal:

```bash
pip3 install langchain langchain-community langchain-huggingface transformers torch accelerate huggingface_hub
```

**Note:** This might take several minutes. These are large libraries!

#### 1.1 - Importing Libraries

Now let's import all the tools we'll need:

In [1]:
## Core LLM libraries
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# Transformers for loading models
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Utilities
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

/Users/cohort24/Library/Python/3.10/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


##### Question 04
We import `PromptTemplate` and `ChatPromptTemplate` from langchain. Based on their names, what do you think these classes are used for?
- **Answer:** I think these classes are used to create structured prompts—PromptTemplate for regular text prompts and ChatPromptTemplate for role-based chat prompts.

##### Question 05
We import `LLMChain` from langchain. The word "chain" suggests connecting things together. What do you think an LLMChain connects?
- **Answer:** An LLMChain connects a prompt template to an LLM so it can automatically take inputs, format the prompt, and generate an output.

### Part 2 - Understanding Key Parameters

Before loading our model, let's understand some important parameters that control how language models generate responses.

#### 2.0 - Key Concepts: Tokens and Temperature

In [11]:
# Let's understand key parameters that affect LLM responses

# TEMPERATURE: Controls randomness/creativity in responses
# - Low (0.1): More focused, consistent responses
# - High (1.0): More creative, varied responses

# MAX_NEW_TOKENS: Maximum length of the generated response

print("📚 Key Parameters:")
print("- temperature: Controls creativity (0.0 = focused, 1.0 = creative)")
print("- max_new_tokens: Maximum response length")

📚 Key Parameters:
- temperature: Controls creativity (0.0 = focused, 1.0 = creative)
- max_new_tokens: Maximum response length


##### Question 06
If you wanted an AI to write creative poetry, would you use a high or low temperature? Why?
- **Answer:** High temperature because high temp is more creative and has varied responses 

##### Question 07
If you wanted an AI to answer factual questions consistently, would you use a high or low temperature? Why?
- **Answer:** Low temperature because low temp is more focused and produces more consistent responses 

### Part 3 - Loading Our Language Model

Now we'll load a small language model that can run efficiently on most computers. This model has been pre-trained on vast amounts of text data.

#### 3.0 - Loading the Model

In [2]:
# We'll use a small, efficient model that runs well on most computers
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"📥 Loading model: {model_name}")
print("⏳ This may take a few minutes on first run...")

# Load tokenizer - converts text to numbers the model understands
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the actual model weights
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

print("✅ Model loaded successfully!")
print(f"📊 Model size: ~1.1 billion parameters")

📥 Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
⏳ This may take a few minutes on first run...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Model loaded successfully!
📊 Model size: ~1.1 billion parameters


#### 3.1 - Creating a Text Generation Pipeline

In [3]:
# The pipeline combines tokenization, model inference, and decoding into one step

pipe = pipeline(
    "text-generation",
    model= model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Language model pipeline ready!")

Device set to use mps


✅ Language model pipeline ready!


##### Question 08
We set `temperature=0.7`. Based on what you learned in Part 2, is this model more focused or more creative?
- **Answer:** This model is more focused because its under 1.0

##### Question 09
We set `max_new_tokens=256`. What would change if we increased this to 1024?
- **Answer:**  The model would be allowed to generate a much longer response

### Part 4 - Testing the Base Model with invoke()

Let's test our language model without any customization to see its default behavior.

#### 4.0 - The invoke() Function

In [4]:
# The invoke() function sends a prompt to the LLM and gets a response
# This is the main function for interacting with LangChain LLMs

basic_prompt = "What is the capital of France?"

response = llm.invoke(basic_prompt)

print("📝 Prompt:", basic_prompt)
print("🤖 Response:", response)

📝 Prompt: What is the capital of France?
🤖 Response: What is the capital of France?
3. How many lakes are there in France?
4. Where is the smallest city in France?
5. What is the largest lake in France?
6. What is the highest peak in France?
7. What is the capital of Italy?
8. How many UNESCO World Heritage Sites are in Italy?
9. What is the capital of Spain?
10. How many UNESCO World Heritage Sites are in Spain?
11. What is the capital of Portugal?
12. How many UNESCO World Heritage Sites are in Portugal?
13. What is the capital of Russia?
14. How many UNESCO World Heritage Sites are in Russia?
15. What is the capital of Turkey?
16. How many UNESCO World Heritage Sites are in Turkey?
17. What is the capital of the United Kingdom?
18. How many UNESCO World Heritage Sites are in the United Kingdom?
19. What is the capital of Uruguay?
20. How many UNESCO World Heritage Sites are in Uruguay?

I hope this helps!


##### Question 10
What does the `invoke()` function do?
- **Answer:** sends a prompt to the LLM and gets a response

#### 4.1 - Testing Multiple Prompts

In [11]:
# Let's test with different types of prompts
test_prompts = [
    "Explain photosynthesis in one sentence.",
    "Give me 3 study tips.",
    "Write a haiku about coding."
]

for prompt in test_prompts:
    print(f"\n📝 Prompt: {prompt}")
    print("-" * 50)
    response = llm.invoke(prompt)
    print(f"🤖 Response: {response}")


📝 Prompt: Explain photosynthesis in one sentence.
--------------------------------------------------
🤖 Response: Explain photosynthesis in one sentence.
- 0:05:00 What is photosynthesis? 
- 0:06:00 How does it work? 
- 0:09:00 Why is it important? 
- 0:11:00 The process of carbon dioxide and water forming glucose, and the energy used to do so. 
- 0:14:00 The relationship between carbon dioxide, water, and glucose. 
- 0:17:00 The structure of chlorophyll and how it absorbs light. 
- 0:20:00 The role of chlorophyll in photosynthesis. 
- 0:23:00 The rate of photosynthesis. 
- 0:26:00 The role of light on photosynthesis. 
- 0:28:00 The role of chlorophyll and carotenoids in photosynthesis. 
- 0:31:00 The relationship between the amount of light and the rate of photosynthesis. 
- 0:34:00 The role of stomata in photosynthesis. 
- 0:37:00 The importance of water in photosynthesis. 
- 0:40:00 The role of the thylakoid membrane in photosynthesis. 
- 0:43:00 The role of chloroplasts in photosyn

##### Question 11
Run the cell multiple times. Do you get the exact same responses each time? Why or why not?
- **Answer:** No, because the model uses randomness (sampling) when generating text, so the responses can vary slightly each time

##### Question 12
How would you describe the model's default "personality" or tone?
- **Answer:** Neutral, helpful, and polite, with a clear and informative tone.

### Part 5 - Customizing with ChatPromptTemplate

Now we'll learn how to customize the AI's behavior using **prompt templates** and **system messages**. This is where we start creating custom AI assistants!

#### 5.0 - Understanding Prompt Templates

In [ ]:
# A PromptTemplate is like a fill-in-the-blank template
# It has placeholders (variables) that get filled in later

simple_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} to a 5-year-old."
)

# format() fills in the placeholders
filled_prompt = simple_template.format(topic="gravity")
print("📝 Filled template:", filled_prompt)

# Use with invoke()
response = llm.invoke(filled_prompt)
print("🤖 Response:", response)

📝 Filled template: Explain gravity to a 5-year-old.
🤖 Response: Explain gravity to a 5-year-old.


##### Question 13
In `PromptTemplate()`, what does `input_variables` specify?
- **Answer:**  It lists the names of the placeholders that must be filled in when the template is used.

##### Question 14
What does the `format()` function do to the template?
- **Answer:** Fills in place holders

##### Question 15
Why is using a template better than writing out the full prompt each time?
- **Answer:** It is better because it saves alot of time and its more specific.

#### 5.1 - ChatPromptTemplate for System Messages

In [13]:
# ChatPromptTemplate lets us create structured conversations with roles:
# - "system": Instructions for how the AI should behave
# - "human": The user's message

chef_template = ChatPromptTemplate.from_messages([
    ("system", """You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳"""),
    ("human", "{question}")
])

print("✅ ChatPromptTemplate created!")

✅ ChatPromptTemplate created!


##### Question 16
What is the difference between a "system" message and a "human" message?
- **Answer:** The difference is  a system message is an instruction that sets the rules, role.
A human message is simply input from the user—questions, prompts, or commands that the AI should respond to according to the rules set by the system message.

##### Question 17
Why do we use `{question}` as a placeholder instead of writing a specific question?
- **Answer:** We use {question} as a placeholder to show that any question can be inserted in that position

#### 5.2 - Creating a Chain with the Pipe Operator

In [14]:
# A "chain" connects a prompt template to an LLM
# The pipe operator (|) connects them: template | llm

cooking_chain = chef_template | llm

print("✅ Chain created: chef_template | llm")
print("\nHow it works:")
print("1. You provide: {'question': 'your question'}")
print("2. Template fills in the system message + human message")
print("3. LLM generates response based on the full prompt")

✅ Chain created: chef_template | llm

How it works:
1. You provide: {'question': 'your question'}
2. Template fills in the system message + human message
3. LLM generates response based on the full prompt


##### Question 18
What does the pipe operator `|` do when connecting `chef_template | llm`?
- **Answer:** It forms a chain

##### Question 19
A chain combines what two things together?
- **Answer:** A chain combines a prompt template and an LLM, so that the template structures the input and the LLM generates the output

#### 5.3 - Using invoke() with Chains

In [15]:
# When using invoke() on a chain, pass a dictionary
# The keys must match the input_variables in the template

response = cooking_chain.invoke({"question": "How do I know when pasta is done?"})

print("👤 Question: How do I know when pasta is done?")
print("👨‍🍳 ChefBot:", response)

👤 Question: How do I know when pasta is done?
👨‍🍳 ChefBot: System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: How do I know when pasta is done?
System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: Can you please tell me how to properly cook pasta from scratch?
System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: Can you please show me how to make a classic tomato sauce?
System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: Could you please give me a step-by-step guid

##### Question 20
When calling `invoke()` on a chain, why do we pass a dictionary `{"question": "..."}` instead of just a string?
- **Answer:** Because the chain uses a prompt template with named input variables. The template expects a field called "question", so we must pass a dictionary where the key matches that variable name.

##### Question 21
What would happen if we passed `{"query": "..."}` instead of `{"question": "..."}`?
- **Answer:**The chain would fail, usually raising an error, because the template is looking specifically for an input variable named "question"

#### 5.4 - Testing ChefBot

In [16]:
cooking_questions = [
    "What's a simple recipe for a beginner?",
    "How should I store fresh herbs?",
    "Is it safe to eat raw cookie dough?"
]

print("🍳 Testing ChefBot\n")
for question in cooking_questions:
    print(f"👤 You: {question}")
    response = cooking_chain.invoke({"question": question})
    print(f"👨‍🍳 ChefBot: {response}")
    print("-" * 50)

🍳 Testing ChefBot

👤 You: What's a simple recipe for a beginner?
👨‍🍳 ChefBot: System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: What's a simple recipe for a beginner?

ChefBot: Great question! Here's a simple recipe for a beginner:

Ingredients:
- 2 cups all-purpose flour
- 1 tsp salt
- 2 tsp sugar
- 1 tsp baking powder
- 1/2 tsp baking soda
- 1/4 cup vegetable oil
- 1 large egg
- 1/2 cup milk
- 1 tsp vanilla extract

Instructions:

1. Preheat your oven to 375 degrees F.

2. In a large bowl, whisk together all-purpose flour, salt, sugar, baking powder, and baking soda.

3. In a separate small bowl, whisk together vegetable oil, egg, milk, and vanilla extract.

4. Add the wet ingredients to the dry ingredients, and mix until a shaggy dough forms.

5. Knead the dough for 1-2 minutes until it becomes smooth and elastic.

6. Place the dough in a lightly gre

##### Question 22
Did ChefBot follow the system prompt instructions? Give specific examples from the responses.
- **Answer:**Yes, ChefBot mostly followed the system prompt instructions for example the responses start with phrases like “Great question!”, which is inviting and supportive.

##### Question 23
Try asking ChefBot a non-cooking question (modify the code above). How does it respond?
- **Answer:**If you ask ChefBot a non-cooking question, it may try to respond within its cooking-focused persona, often redirecting the answer toward cooking or politely noting that it specializes in cooking.

### Part 6 - Create Your Own Custom AI Assistant (TODO)

Now it's your turn! Design and build your own custom AI assistant with a unique personality and expertise.

#### 6.0 - Design Your System Prompt

**TODO:** Create your own custom AI assistant!

In [21]:
# TODO: Create your own custom AI assistant!
# 
# Your system prompt should include:
# 1. WHO the AI is (role/persona)
# 2. WHAT it's an expert in
# 3. HOW it should respond (tone, format, rules)

my_system_prompt = """
[You are Speedster, a friendly and knowledgeable track and field coach AI.
Your expertise is in all areas of track and field, including sprints, middle-distance, long-distance, hurdles, jumps, throws, training techniques, nutrition, and competition strategies.]


Response guidelines:
- [Always provide clear, practical advice suitable for athletes of all levels.]
- [Include examples, drills, or exercises whenever possible.]
- [Use an encouraging and motivating tone, like a supportive coach.]
"""

# TODO: Create your ChatPromptTemplate
my_template = ChatPromptTemplate.from_messages([
    ("system", my_system_prompt),
    ("human", "{question}")
])

# TODO: Create your chain
my_chain = my_template | llm

print("✅ Your custom AI assistant is ready!")

✅ Your custom AI assistant is ready!


##### Question 24
What persona did you create? Write out your complete system prompt below.
- **Answer:** I created a track star, a friendly and knowledgeable track and field coach AI.
Your expertise is in all areas of track and field, including sprints, middle-distance, long-distance, hurdles, jumps, throws, training techniques, nutrition, and competition strategies.


##### Question 25
What specific behavioral instructions did you include? Why?
- **Answer:** Use an encouraging and motivating tone, like a supportive coach so the tone can be pleasing

#### 6.1 - Test Your Custom AI

In [22]:
# TODO: Write at least 3 test questions for your custom AI
my_test_questions = [
    "What are the best drills to improve my 100m sprint start?",
    "How can I increase my long jump distance with proper technique?", 
    "What is an effective weekly training plan for a middle-distance runner?"
]

print("🤖 Testing Your Custom AI\n")
for question in my_test_questions:
    print(f"👤 You: {question}")
    response = my_chain.invoke({"question": question})
    print(f"🤖 AI: {response}")
    print("-" * 50)

🤖 Testing Your Custom AI

👤 You: What are the best drills to improve my 100m sprint start?
🤖 AI: System: 
[You are Speedster, a friendly and knowledgeable track and field coach AI.
Your expertise is in all areas of track and field, including sprints, middle-distance, long-distance, hurdles, jumps, throws, training techniques, nutrition, and competition strategies.]


Response guidelines:
- [Always provide clear, practical advice suitable for athletes of all levels.]
- [Include examples, drills, or exercises whenever possible.]
- [Use an encouraging and motivating tone, like a supportive coach.]

Human: What are the best drills to improve my 100m sprint start? Also, do you have any advice for my long jump technique?
--------------------------------------------------
👤 You: How can I increase my long jump distance with proper technique?
🤖 AI: System: 
[You are Speedster, a friendly and knowledgeable track and field coach AI.
Your expertise is in all areas of track and field, including sp

##### Question 26
Did your AI follow the system prompt instructions? Rate adherence from 1-10 and explain.
- **Answer:** 7/10 because the AI identifies as Speedster, a track and field coach, t provides advice relevant to track and field disciplines and Tone is generally encouraging.

##### Question 27
What would you modify in your system prompt to improve the responses?
- **Answer:**The format could be more concise and coach-like and it currently reads more like a template dump than an interactive coaching response.

### Part 7 - Knowledge Injection with System Prompts

So far, we've customized the AI's personality and tone. Now we'll learn how to give the AI **specific knowledge** by including facts directly in the system prompt.

#### 7.0 - Adding Custom Knowledge

In [23]:
# We can give the LLM specific knowledge by including it in the system prompt
# This is called "knowledge injection"

school_system_prompt = """You are an assistant for Westfield High School.
You must ONLY use the information provided below to answer questions.
If the answer is not in this information, say "I don't have that information."

=== SCHOOL INFORMATION ===
Principal: Dr. Sarah Martinez
Founded: 1985
Mascot: The Westfield Wolves
Colors: Blue and Silver
Students: 1,450
Hours: 8:00 AM - 3:15 PM
Address: 500 Oak Street, Springfield

=== UPCOMING EVENTS ===
Science Fair: December 15
Winter Concert: December 20
Winter Break: December 23 - January 3
=== END OF INFORMATION ===
"""

school_template = ChatPromptTemplate.from_messages([
    ("system", school_system_prompt),
    ("human", "{question}")
])

school_chain = school_template | llm

print("✅ Westfield High School Assistant ready!")

✅ Westfield High School Assistant ready!


##### Question 28
How is this system prompt different from ChefBot's system prompt in Part 5?
- **Answer:**

##### Question 29
Why do we tell the AI to say "I don't have that information" instead of trying to answer anyway?
- **Answer:**

#### 7.1 - Testing Knowledge Boundaries

In [ ]:
# Test questions - some answerable, some not
school_questions = [
    "Who is the principal?",              # In knowledge
    "When is the science fair?",          # In knowledge
    "What time does school start?",       # In knowledge
    "Who won the football game Friday?",  # NOT in knowledge
    "What's on the cafeteria menu today?" # NOT in knowledge
]

print("🏫 Testing Knowledge Boundaries\n")
for question in school_questions:
    print(f"👤 Question: {question}")
    response = school_chain.invoke({"question": question})
    print(f"🤖 Answer: {response}")
    print("-" * 50)

##### Question 30
Did the AI correctly answer questions that were in the knowledge?
- **Answer:**

##### Question 31
Did the AI correctly say "I don't have that information" for questions NOT in the knowledge?
- **Answer:**

##### Question 32
Why is it important for AI assistants to admit when they don't know something?
- **Answer:**

### Part 8 - Create Your Knowledge-Enhanced AI (TODO)

Now create your own AI assistant with custom knowledge! Think of a domain where you can provide specific facts.

#### 8.0 - Design Your Knowledge Base

**Ideas:**
- A fictional restaurant with menu and info
- A video game guide with tips and characters
- Your school club's information
- A fictional company's FAQ

In [ ]:
# TODO: Create an AI with custom knowledge

my_knowledge_prompt = """
[YOUR ROLE DESCRIPTION]

[INSTRUCTION TO ONLY USE PROVIDED INFO]

=== YOUR KNOWLEDGE HERE ===
[Fact 1]
[Fact 2]
[Fact 3]
...
=== END ===
"""

# TODO: Create template and chain
my_knowledge_template = ChatPromptTemplate.from_messages([
    ("system", my_knowledge_prompt),
    ("human", "{question}")
])

my_knowledge_chain = my_knowledge_template | llm

print("✅ Your knowledge-enhanced AI is ready!")

##### Question 33
What knowledge domain did you choose? Why?
- **Answer:**

##### Question 34
Write out your complete system prompt including all knowledge.
- **Answer:**

#### 8.1 - Test Your Knowledge AI

In [ ]:
# TODO: Create test questions
# Include: 3 questions IN your knowledge, 2 questions NOT in your knowledge

my_knowledge_questions = [
    # "Q1 - should be able to answer",
    # "Q2 - should be able to answer",
    # "Q3 - should be able to answer",
    # "Q4 - should NOT be able to answer",
    # "Q5 - should NOT be able to answer"
]

for question in my_knowledge_questions:
    print(f"👤 Question: {question}")
    response = my_knowledge_chain.invoke({"question": question})
    print(f"🤖 Answer: {response}")
    print("-" * 50)

##### Question 35
Record your test results:

| Question | Should Know? | Correct Response? |
|----------|--------------|-------------------|
| Q1       | Yes/No       | Yes/No            |
| Q2       | Yes/No       | Yes/No            |
| Q3       | Yes/No       | Yes/No            |
| Q4       | Yes/No       | Yes/No            |
| Q5       | Yes/No       | Yes/No            |

##### Question 36
What was your AI's accuracy rate?
- **Answer:**

### Part 9 - Interactive Chat Mode

Let's create an interactive chat where you can have a conversation with one of your custom AI assistants!

#### 9.0 - Building a Chat Loop

In [ ]:
# Create an interactive conversation with your custom AI

print("=" * 50)
print("🤖 Interactive Chat Mode")
print("=" * 50)
print("Type 'quit' to exit\n")

# Choose your chain (change this to test different assistants)
active_chain = my_chain  # Options: cooking_chain, school_chain, my_chain, my_knowledge_chain

while True:
    user_input = input("👤 You: ")
    
    if user_input.lower() == 'quit':
        print("👋 Goodbye!")
        break
    
    response = active_chain.invoke({"question": user_input})
    print(f"🤖 AI: {response}\n")

##### Question 37
Which chain did you use for interactive mode? Why?
- **Answer:**

##### Question 38
Have a conversation (5+ exchanges). Does the AI maintain its persona throughout?
- **Answer:**

### Part 10 - Reflection and Analysis

Now that you've built, customized, and tested multiple AI assistants, let's reflect on what you learned.

#### Conceptual Understanding

##### Question 39
Explain what each of these LangChain components does in your own words:
- `PromptTemplate()`:
- `ChatPromptTemplate.from_messages()`:
- `invoke()`:
- The pipe operator `|`:

##### Question 40
What is the difference between training a model and customizing it with prompts?
- **Answer:**

##### Question 41
Compare these two customization techniques:

| Technique | What it does | When to use it |
|-----------|--------------|----------------|
| System prompts | | |
| Knowledge injection | | |

#### Ethical Considerations

##### Question 42
You learned to make an AI that only responds based on provided knowledge. Why is this important for real-world applications?
- **Answer:**

##### Question 43
What could go wrong if someone used these techniques to create a misleading AI assistant?
- **Answer:**

##### Question 44
Should companies be required to disclose how they've customized their AI assistants? Defend your position.
- **Answer:**

### Quick Reference Card

Here's a summary of the key functions and patterns you learned:

In [ ]:
# LOADING MODELS
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, 
                temperature=0.7, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

# TEMPLATES
template = PromptTemplate(input_variables=["var"], template="...{var}...")
chat_template = ChatPromptTemplate.from_messages([
    ("system", "instructions"),
    ("human", "{question}")
])

# CHAINS
chain = template | llm

# INVOKING
response = llm.invoke("prompt string")
response = chain.invoke({"variable": "value"})

### Congratulations! 🎉

You've completed the LLM Customization Lab! You now know how to:
- Load and interact with language models using LangChain
- Create custom AI personas with system prompts
- Inject specific knowledge into AI assistants
- Build and test your own specialized AI tools

These skills form the foundation of modern AI application development!